In [1]:
import subprocess
import json
import sys
sys.path.append("../src/")
from utils import get_metrics
import numpy as np

This assumes that the benchmarks have been generated using the `../src/create_benchmark_data.py`. For the sake of convenience, the binary has been renamed to just `process_trajectories`

In [2]:
def run_eval_tool(input_file, output_file, detection_window, snooze_window, num_threshold=10000):
    subprocess.run(['../src/process_trajectories', '-i', input_file, '-o', output_file, '-d', str(detection_window), '-s', str(snooze_window), '-n', str(num_threshold)], stdout=subprocess.DEVNULL)

### Read in data 

In [3]:
with open("../benchmark_data/benchmark_trajectories_200.json", "r") as f:
    benchmark_200 = json.load(f)

with open("../benchmark_data/benchmark_trajectories_2000.json", "r") as f:
    benchmark_2000 = json.load(f)
    
with open("../benchmark_data/benchmark_trajectories_20000.json", "r") as f:
    benchmark_20000 = json.load(f)

## Using python implementation 

In [4]:
get_metrics(benchmark_200, thresholds=np.linspace(0,1, 100), snooze_window=5, detection_window=12)[0:2]

[{'threshold': 0.0,
  'episode_tp': 91,
  'episode_fp': 108,
  'episode_tn': 0,
  'episode_fn': 1,
  'prediction_tp': 211,
  'prediction_fp': 7400},
 {'threshold': 0.010101010101010102,
  'episode_tp': 91,
  'episode_fp': 108,
  'episode_tn': 0,
  'episode_fn': 1,
  'prediction_tp': 211,
  'prediction_fp': 7397}]

## Using native implementation

In [5]:
run_eval_tool(input_file="../benchmark_data/benchmark_trajectories_200.json",
              output_file="../benchmark_data/tmp_results.json",
              snooze_window=5,
              detection_window=12,
              num_threshold=100)

In [6]:
with open("../benchmark_data/tmp_results.json", "r") as f:
    example_results = json.load(f)

In [7]:
example_results[0:2]

[{'threshold': 0.0,
  'episode_tp': 91,
  'episode_fp': 108,
  'episode_tn': 0,
  'episode_fn': 1,
  'prediction_tp': 211,
  'prediction_fp': 7400},
 {'threshold': 0.0101010101010101,
  'episode_tp': 91,
  'episode_fp': 108,
  'episode_tn': 0,
  'episode_fn': 1,
  'prediction_tp': 211,
  'prediction_fp': 7397}]

# Example Benchmarking 

In [8]:
%%timeit -n 1 -r 1
get_metrics(benchmark_2000, thresholds=np.linspace(0,1, 1000), snooze_window=5, detection_window=12)[0:2]

1min 52s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
%%timeit -n 1 -r 1
run_eval_tool(input_file="../benchmark_data/benchmark_trajectories_2000.json",
              output_file="../benchmark_data/tmp_results.json",
              snooze_window=5,
              detection_window=12,
              num_threshold=1000)

10.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%%timeit -n 1 -r 1
get_metrics(benchmark_20000, thresholds=np.linspace(0,1, 1000), snooze_window=5, detection_window=12)[0:2]

18min 58s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
%%timeit -n 1 -r 1
run_eval_tool(input_file="../benchmark_data/benchmark_trajectories_20000.json",
              output_file="../benchmark_data/tmp_results.json",
              snooze_window=5,
              detection_window=12,
              num_threshold=1000)

1min 49s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
